In [63]:
import nltk
import re

In [64]:
sents = ['Вася читает мою книгу', 'Напиши какое-нибудь письмо',
         'Этот веселый мальчик идет', 'Он любит читать всякие книги']

In [65]:
rules = """
    S -> NP VP | NP V | VP
    NP -> ADJ NP | ADJ N | NN | N
    VP -> V NP | V VP | V
    NN -> 'Вася'
    N -> 'книгу' | 'письмо' | 'мальчик' | 'Он' | 'книги'
    V -> 'читает' | 'Напиши' | 'идет' | 'любит' | 'читать'
    ADJ -> 'мою' | 'какое-нибудь' | 'Этот' | 'веселый' | 'всякие'
""".split('\n')

In [66]:
grammar = nltk.CFG.fromstring('\n'.join(rules))

In [67]:
def print_parses(parser, sentence):
    for tree in parser.parse(sentence.split()):
        print(tree)
        print()

# Алгоритм Кока-Янгера-Касами (CYK)


In [68]:
S = 'Вася читает мою книгу'

In [69]:
import pandas as pd
data = [['S', '', '', ''], 
        ['', 'VP', '', ''], 
        ['NP', '', 'NP', ''], 
        ['NN', 'V', 'ADJ', 'N'], 
        ['Вася', 'читает', 'мою', 'книгу']]
pd.DataFrame(data)

,0,1,2,3
0,S,,,
1,,VP,,
2,NP,,NP,
3,NN,V,ADJ,N
4,Вася,читает,мою,книгу


Сработавшие правила:  
    1)NN -> Вася, V -> читает, ADJ -> мою, N -> книгу  
    2)NP -> NN, NP -> ADJ N  
    3)VP -> V NP  
    4)S -> VP NP  

# Алгоритм Эрли

За основу взята 13 глава учебника Jurafsky + Martin (2 изд.). 

In [70]:
S = 'Вася читает мою книгу'

In [71]:
#Chart[0]
data = [['[0:0]', 'S -> * NP VP', 'predictor'], 
        ['[0:0]', 'S -> * NP V', 'predictor'], 
        ['[0:0]', 'S -> * VP', 'predictor'],
        ['[0:0]', 'VP -> * V NP', 'predictor'], 
        ['[0:0]', 'VP -> * V VP', 'predictor'],
        ['[0:0]', 'VP -> * V', 'predictor'],
        ['[0:0]', 'NP -> * ADJ NP', 'predictor'],
        ['[0:0]', 'NP -> * ADJ N', 'predictor'],
        ['[0:0]', 'NP -> * NN', 'predictor'],
        ['[0:0]', 'NP -> * N', 'predictor'],]
pd.DataFrame(data, columns = ['from to', 'item', 'function'])

,from to,item,function
0,[0:0],S -> * NP VP,predictor
1,[0:0],S -> * NP V,predictor
2,[0:0],S -> * VP,predictor
3,[0:0],VP -> * V NP,predictor
4,[0:0],VP -> * V VP,predictor
5,[0:0],VP -> * V,predictor
6,[0:0],NP -> * ADJ NP,predictor
7,[0:0],NP -> * ADJ N,predictor
8,[0:0],NP -> * NN,predictor
9,[0:0],NP -> * N,predictor


In [72]:
#Chart[1]
data = [['[0:1]', '''N -> 'Вася' *''', 'scanner'], 
        ['[0:1]', 'NP -> NN *', 'completer'], 
        ['[0:1]', 'S -> * VP', 'completer'],
        ['[0:1]', 'S  -> NP * VP', 'completer'], 
        ['[1:1]', 'VP -> * V NP', 'predictor'],
        ['[1:1]', 'VP -> * V VP', 'predictor'],
        ['[1:1]', 'VP -> * V', 'predictor'],]
pd.DataFrame(data, columns = ['from to', 'item', 'function'])

,from to,item,function
0,[0:1],N -> 'Вася' *,scanner
1,[0:1],NP -> NN *,completer
2,[0:1],S -> * VP,completer
3,[0:1],S -> NP * VP,completer
4,[1:1],VP -> * V NP,predictor
5,[1:1],VP -> * V VP,predictor
6,[1:1],VP -> * V,predictor


In [73]:
#Chart[2]
data = [['[1:2]', '''V -> 'читает' *''', 'scanner'], 
        ['[0:2]', 'S  -> NP VP *', 'completer'],
        ['[1:2]', 'VP -> V * NP', 'completer'], 
        ['[1:2]', 'VP -> V * VP', 'completer'], 
        ['[0:2]', 'S  -> NP VP *', 'completer'],
        ['[2:2]', 'VP -> * V NP', 'predictor'],
        ['[2:2]', 'VP -> * V VP', 'predictor'],
        ['[2:2]', 'VP -> * V', 'predictor'],
        ['[2:2]', 'NP -> * ADJ NP', 'predictor'],
        ['[2:2]', 'NP -> * ADJ N', 'predictor'],
        ['[2:2]', 'NP -> * NN', 'predictor'],
        ['[2:2]', 'NP -> * N', 'predictor']]
pd.DataFrame(data, columns = ['from to', 'item', 'function'])

,from to,item,function
0,[1:2],V -> 'читает' *,scanner
1,[0:2],S -> NP VP *,completer
2,[1:2],VP -> V * NP,completer
3,[1:2],VP -> V * VP,completer
4,[0:2],S -> NP VP *,completer
5,[2:2],VP -> * V NP,predictor
6,[2:2],VP -> * V VP,predictor
7,[2:2],VP -> * V,predictor
8,[2:2],NP -> * ADJ NP,predictor
9,[2:2],NP -> * ADJ N,predictor


In [74]:
#Chart[3]
data = [['[2:3]', '''ADJ -> 'мою' *''', 'scanner'], 
        ['[2:3]', 'NP -> ADJ * NP', 'completer'],
        ['[2:3]', 'NP -> ADJ * N', 'completer'],
        ['[3:3]', 'NP -> * ADJ NP', 'predictor'],
        ['[3:3]', 'NP -> * ADJ N', 'predictor'],
        ['[3:3]', 'NP -> * NN', 'predictor'],
        ['[3:3]', 'NP -> * N', 'predictor']]
pd.DataFrame(data, columns = ['from to', 'item', 'function'])

,from to,item,function
0,[2:3],ADJ -> 'мою' *,scanner
1,[2:3],NP -> ADJ * NP,completer
2,[2:3],NP -> ADJ * N,completer
3,[3:3],NP -> * ADJ NP,predictor
4,[3:3],NP -> * ADJ N,predictor
5,[3:3],NP -> * NN,predictor
6,[3:3],NP -> * N,predictor


In [75]:
#Chart[3]
data = [['[3:4]', '''N  -> 'книгу' *''', 'scanner'], 
        ['[2:4]', 'NP -> ADJ NP *', 'completer'],
        ['[2:4]', 'NP -> ADJ N *', 'completer'],
        ['[3:4]', 'NP -> N *', 'completer'],
        ['[1:4]', 'VP -> V NP *', 'completer'],
        ['[0:4]', 'S -> V NP *', 'completer'],
        ['[1:4]', 'VP -> V NP *', 'ompleter']]
pd.DataFrame(data, columns = ['from to', 'item', 'function'])

,from to,item,function
0,[3:4],N -> 'книгу' *,scanner
1,[2:4],NP -> ADJ NP *,completer
2,[2:4],NP -> ADJ N *,completer
3,[3:4],NP -> N *,completer
4,[1:4],VP -> V NP *,completer
5,[0:4],S -> V NP *,completer
6,[1:4],VP -> V NP *,ompleter


# Проверим результаты работы с помощью NLTK

In [76]:
cp = nltk.EarleyChartParser(grammar)
for i in range(len(sents)):
    print(sents[i])
    print_parses(cp, sents[i])

Вася читает мою книгу
(S (NP (NN Вася)) (VP (V читает) (NP (ADJ мою) (NP (N книгу)))))

(S (NP (NN Вася)) (VP (V читает) (NP (ADJ мою) (N книгу))))

Напиши какое-нибудь письмо
(S (VP (V Напиши) (NP (ADJ какое-нибудь) (NP (N письмо)))))

(S (VP (V Напиши) (NP (ADJ какое-нибудь) (N письмо))))

Этот веселый мальчик идет
(S (NP (ADJ Этот) (NP (ADJ веселый) (NP (N мальчик)))) (V идет))

(S (NP (ADJ Этот) (NP (ADJ веселый) (N мальчик))) (V идет))

(S (NP (ADJ Этот) (NP (ADJ веселый) (NP (N мальчик)))) (VP (V идет)))

(S (NP (ADJ Этот) (NP (ADJ веселый) (N мальчик))) (VP (V идет)))

Он любит читать всякие книги
(S
  (NP (N Он))
  (VP (V любит) (VP (V читать) (NP (ADJ всякие) (NP (N книги))))))

(S
  (NP (N Он))
  (VP (V любит) (VP (V читать) (NP (ADJ всякие) (N книги)))))



Мы видим, что для каждого предложения, парсер выдает неоднозначные результаты. Связано это с разделением NP и VP. 

# Улучшения

Значительным улучшением работы парсера может быть использование морфологического анализатора. Во первых, с его помощью можно обрабатывать новые слова (в одной функции можно определять часть речи нового слова и создавать новое правило). Во-вторых, использование морфологического анализатора может упроить работу в том плане, что мы можем записывать в грамматику только начальную форму слова, а перед парсингом приводить слова из предложения в их начальную форму. Это сократит размер грамматики в разы.

Также, возможно стоит изменить саму грамматику, чтобы не возникало неоднозначностей, связанных с VP: VP -> V. И изменим NP -> NN | N только на NN, добавив NN -> 'Он'

In [77]:
new_rules = """
    S -> NP VP | NP V | VP
    NP -> ADJ NP | ADJ N | NN
    VP -> V NP | V VP
    NN -> 'Вася' | 'Он'
    N -> 'книгу' | 'письмо' | 'мальчик' | 'Он' | 'книги'
    V -> 'читает' | 'Напиши' | 'идет' | 'любит' | 'читать'
    ADJ -> 'мою' | 'какое-нибудь' | 'Этот' | 'веселый' | 'всякие'
""".split('\n')

In [78]:
new_grammar = nltk.CFG.fromstring('\n'.join(new_rules))

In [79]:
new_cp = nltk.EarleyChartParser(new_grammar)
for i in range(len(sents)):
    print(sents[i])
    print_parses(new_cp, sents[i])

Вася читает мою книгу
(S (NP (NN Вася)) (VP (V читает) (NP (ADJ мою) (N книгу))))

Напиши какое-нибудь письмо
(S (VP (V Напиши) (NP (ADJ какое-нибудь) (N письмо))))

Этот веселый мальчик идет
(S (NP (ADJ Этот) (NP (ADJ веселый) (N мальчик))) (V идет))

Он любит читать всякие книги
(S
  (NP (NN Он))
  (VP (V любит) (VP (V читать) (NP (ADJ всякие) (N книги)))))



# Неоднозначный разбор

Помимо того, что уже есть, неоднозначный разбор будет если разбирать слово, которое может относиться к двум частям речи. Например, суши может быть глаголом (суши весла), а может быть существительным (мы ели вкусные суши) 